# Fit the Model

In [105]:
%%sh
drum fit --code-dir ./model \
--input data/10K_Lending_Club_Loans_utf-8.csv \
--output ./model \
--target-type binary \
--target is_bad \
--docker env

Files were overwritten: {'/opt/fit_output_dir/create_data.py', '/opt/fit_output_dir/custom_model.py', '/opt/fit_output_dir/feature_detail.yaml', '/opt/fit_output_dir/model-config.yaml', '/opt/fit_output_dir/custom.py', '/opt/fit_output_dir/README.md', '/opt/fit_output_dir/create_pipeline.py', '/opt/fit_output_dir/schema.json', '/opt/fit_output_dir/artifact.pkl'}
Success 🎉
Host DRUM version: 1.4.4
Container DRUM version: 
the input device is not a TTY



# Testing the model

## Performance Testing

In [106]:
%%sh
drum perf-test \
--code-dir ./model \
--input "data/10K_LC_short.csv" \
--target-type binary \
--positive-class-label 1 \
--negative-class-label 0 \
--docker env \
--verbose

Detected perf-test mode
DRUM performance test
Model:      /Users/timothy.whittaker/Desktop/git/ford-mlops/model
Data:       /Users/timothy.whittaker/Desktop/git/ford-mlops/data/10K_LC_short.csv
# Features: 35
>78Preparing test data...
Running drum using: [drum server --code-dir /Users/timothy.whittaker/Desktop/git/ford-mlops/model --address localhost:59718 --logging-level warning --show-perf --target-type binary --positive-class-label 1 --negative-class-label 0 --docker env]



Running test case with timeout: 180
Running test case: 550 bytes - 1 samples, 100 iterations
Running test case with timeout: 180
Running test case: 0.1MB - 190 samples, 50 iterations
Running test case with timeout: 180
Running test case: 10MB - 19051 samples, 5 iterations
Running test case with timeout: 180
Running test case: 50MB - 95256 samples, 1 iterations
Test is done stopping drum server

 size    sample   iters    min     avg     max    contai   conta   conta   total
           s                       

## Validation

In [107]:
%%sh
drum validation \
--code-dir ./model \
--input "data/10K_LC_short.csv" \
--target-type binary \
--positive-class-label 1 \
--negative-class-label 0 \
--docker env > validation.log

docker: Error response from daemon: invalid mode: /opt/input.csv.
See 'docker run --help'.
2020-12-16 20:45:18,951 ERROR drum:  Error from docker process: 125
Traceback (most recent call last):
  File "/usr/local/bin/drum", line 6, in <module>
    main()
  File "/usr/local/lib/python3.7/site-packages/datarobot_drum/drum/main.py", line 96, in main
    CMRunner(runtime).run()
  File "/usr/local/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 289, in run
    self._run_fit_and_predictions_pipelines_in_mlpiper()
  File "/usr/local/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 559, in _run_fit_and_predictions_pipelines_in_mlpiper
    _pipeline_executor.run_pipeline(cleanup=False)
  File "/usr/local/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 261, in run_pipeline
    self._run_pipeline()
  File "/usr/local/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 251, in _run_pipeline
    self._dag.run_connected_pipeline(self._ml_engin

In [109]:
! tail -n 30 validation.log

the input device is not a TTY



Validation checks results
      Test case         Status                       Details                    
Null value imputation   FAILED   Null value imputation check performs check by  
                                 imputing each feature with NaN value. If check 
                                 fails for a feature, test dataset is saved in  
                                 /tmp/drum_validation_checks_. Make sure to     
                                 delete those folders if it takes too much      
                                 space.                                         
                                                                                
                                 Failed feature                          Dataset
                                 filename                                       
                                 Unnamed: 0       /tmp/drum_validation_checks_s4
                                 s9yat0/null_value

## Batch Scoring

In [110]:
%%sh
drum score \
--code-dir ./model \
--target-type binary \
--positive-class-label pos_class_prediction \
--negative-class-label neg_class_prediction \
--input data/10K_LC_short.csv \
--docker env \
--verbose --output data/predictions.csv

Detected score mode
Detected /opt/model/custom.py .. trying to load hooks
Finished loading model, total used 100 iterations
 
 
Component: generic_predictor
Language:  Python
Output:
------------------------------------------------------------
{'target_type': <TargetType.BINARY: 'binary'>, 'positive_class_label': 'pos_class_prediction', 'negative_class_label': 'neg_class_prediction'}
------------------------------------------------------------
Runtime:    0.1 sec
NR outputs: 0
 
docker command: [docker run --rm --interactive --user $(id -u):$(id -g)   -v /Users/timothy.whittaker/Desktop/git/ford-mlops/model:/opt/model -v "/Users/timothy.whittaker/Desktop/git/ford-mlops/data/10K_LC_short.csv":/opt/input.csv -v "/Users/timothy.whittaker/Desktop/git/ford-mlops/data/predictions.csv":/opt/output.csv sha256:d796fdc6e00d9b1b32519ccd3e39b93d648769b66bdbdf255aa4f33a3ca24b37 drum score --code-dir /opt/model --target-type binary --positive-class-label pos_class_prediction --negative-class-label n

In [111]:
!head -n 5 data/predictions.csv

neg_class_prediction,pos_class_prediction
0.9816415360226007,0.01835846397739924
0.8722987977456673,0.12770120225433262
0.9798190190554988,0.020180980944501145
0.9299558044221623,0.07004419557783767


## Batch Scoring Unstructured Model

Returns Predictions and SHAP Values

In [112]:
%%sh
drum score \
--code-dir ./model \
--input "data/10K_LC_short.csv" \
--target-type unstructured \
--content-type application/text \
--docker env \
--verbose --output data/predictions.json

Detected score mode
Detected /opt/model/custom.py .. trying to load hooks
Finished loading model, total used 100 iterations
 
 
Component: generic_predictor
Language:  Python
Output:
------------------------------------------------------------
Incoming content type params:  {'mimetype': 'application/text'}
Incoming data type:  <class 'bytes'>
Incoming query params:  {}
------------------------------------------------------------
Runtime:    0.2 sec
NR outputs: 0
 
docker command: [docker run --rm --interactive --user $(id -u):$(id -g)   -v /Users/timothy.whittaker/Desktop/git/ford-mlops/model:/opt/model -v "/Users/timothy.whittaker/Desktop/git/ford-mlops/data/10K_LC_short.csv":/opt/input.csv -v "/Users/timothy.whittaker/Desktop/git/ford-mlops/data/predictions.json":/opt/output.csv sha256:d796fdc6e00d9b1b32519ccd3e39b93d648769b66bdbdf255aa4f33a3ca24b37 drum score --code-dir /opt/model --input /opt/input.csv --target-type unstructured --content-type application/text --verbose --output /o

LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


In [126]:
import json
import pprint
d = json.load(open("data/predictions.json", "rb"))
pprint.pprint(d["data"][0])

{'predictions': {'negative_class_label_prob': 0.9816415360226007,
                 'positive_class_label_prob': 0.01835846397739924},
 'shap_values': {'POS_CLASS_SHAP_addr_state': 0.008390547232085098,
                 'POS_CLASS_SHAP_addr_state_name': -0.01587483936197775,
                 'POS_CLASS_SHAP_annual_inc': 0.08800024892580516,
                 'POS_CLASS_SHAP_delinq_2yrs': -0.0036656212456457485,
                 'POS_CLASS_SHAP_dti': 0.047127242530700095,
                 'POS_CLASS_SHAP_earliest_cr_line day of week': -0.04021584397331488,
                 'POS_CLASS_SHAP_earliest_cr_line month of year': -0.17162178620756163,
                 'POS_CLASS_SHAP_emp_length': -0.1137186917067495,
                 'POS_CLASS_SHAP_emp_title': -0.02926689870639889,
                 'POS_CLASS_SHAP_funded_amnt': 0.017315288502232875,
                 'POS_CLASS_SHAP_grade': 0.02069466102221773,
                 'POS_CLASS_SHAP_home_ownership': -0.01047770841421593,
               

## Serving Model

In [114]:
import subprocess
import requests
import time

In [117]:
run_inference_server = [
    "drum", "server",
    "--code-dir","./model", 
    "--address", "0.0.0.0:6789", 
    "--show-perf",
    "--target-type", "binary",
    "--logging-level", "info",
    "--show-stacktrace",
    "--verbose", 
    "--docker", "env"
]
inference_server = subprocess.Popen(run_inference_server, stdout=subprocess.PIPE)

In [118]:
## snoozing before pinging the server to give it time to actually start
time.sleep(5)
print('check status')
requests.request("GET", "http://0.0.0.0:6789/").content

check status


b'{"message":"OK"}\n'

In [119]:
## shutdown
requests.request("POST", "http://0.0.0.0:6789/shutdown/").content

b'Server shutting down...'

In [120]:
inference_server.terminate()
inference_server.stdout.readlines()

[b'Detected REST server mode - this is an advanced option\n',
 b'Detected /opt/model/custom.py .. trying to load hooks\n',
 b'Finished loading model, total used 100 iterations\n',
 b'\x1b \x1b\n',
 b'\x1b \x1b\n',
 b'\x1b============================================================\x1b\n',
 b'\x1bComponent: prediction_server\x1b\n',
 b'\x1bLanguage:  Python\x1b\n',
 b'\x1bOutput:\x1b\n',
 b'\x1b------------------------------------------------------------\x1b\n',
 b' * Serving Flask app "datarobot_drum.drum.server" (lazy loading)\n',
 b' * Environment: production\n',
 b'   WARNING: This is a development server. Do not use it in a production deployment.\n',
 b'   Use a production WSGI server instead.\n',
 b' * Debug mode: off\n',
 b'docker command: [docker run --rm --interactive --user $(id -u):$(id -g)   -v /Users/timothy.whittaker/Desktop/git/ford-mlops/model:/opt/model -p 6789:6789 sha256:d796fdc6e00d9b1b32519ccd3e39b93d648769b66bdbdf255aa4f33a3ca24b37 drum server --code-dir /opt/model

## Push Artifacts and Environment to DataRobot

In [121]:
!python push.py --env-dir ./env --model-dir ./model --logging-level INFO --max-wait 600 --update-env True

Namespace(env_dir='./env', logging_level='INFO', max_wait=600, model_dir='./model', update_env=True)
__main__ - INFO - 2020-12-16 21:04:30,492 - Namespace(env_dir='./env', logging_level='INFO', max_wait=600, model_dir='./model', update_env=True)
__main__ - INFO - 2020-12-16 21:04:30,928 - load model config
__main__ - INFO - 2020-12-16 21:04:30,933 - {'description': None, 'language': 'Python', 'majorVersion': True, 'name': 'lgbm-classifier', 'negativeClassLabel': 0, 'positiveClassLabel': 1, 'predictionThreshold': 0.17, 'targetName': 'is_bad', 'targetType': 'binary', 'testData': '/Users/timothy.whittaker/Desktop/git/ford-mlops/data/10K_LC_short.csv', 'trainingData': '/Users/timothy.whittaker/Desktop/git/ford-mlops/data/10K_Lending_Club_Loans_utf-8.csv', 'type': 'inference', 'versionId': None, 'environmentID': '5fda0b23d33aebb30a492f74', 'environmentVersionID': '5fda0b242d986b76e2d33bf8'}
__main__ - INFO - 2020-12-16 21:04:30,933 - create new inference model
__main__ - INFO - 2020-12-16 2

# Test Model in DataRobot

In [122]:
!python test.py --model-dir ./model --logging-level INFO --max-wait 600

Namespace(logging_level='INFO', max_wait=600, model_dir='./model')
__main__ - INFO - 2020-12-16 21:05:38,784 - Namespace(logging_level='INFO', max_wait=600, model_dir='./model')
__main__ - INFO - 2020-12-16 21:05:38,785 - loading model config
__main__ - INFO - 2020-12-16 21:05:38,788 - loading dataset to ai catalog
__main__ - INFO - 2020-12-16 21:06:21,218 - starting custom model test
__main__ - INFO - 2020-12-16 21:11:01,712 - test complete
__main__ - INFO - 2020-12-16 21:11:01,713 - warning 😱
__main__ - ERROR - 2020-12-16 21:11:01,713 - Test: error_check
__main__ - ERROR - 2020-12-16 21:11:01,713 - Status: succeeded
__main__ - ERROR - 2020-12-16 21:11:01,713 - Message: 
__main__ - ERROR - 2020-12-16 21:11:01,713 - Test: null_value_imputation
__main__ - ERROR - 2020-12-16 21:11:01,713 - Status: warning
__main__ - ERROR - 2020-12-16 21:11:01,713 - Message: Model cannot impute null values for the following columns:
int_rate, zip_code, addr_state
__main__ - ERROR - 2020-12-16 21:11:01,71